#### There should be a day for each panel. Make sure there is an hour for each day

In [5]:
import pandas as pd
import numpy  as np
import time
from config import getClient
from checkMissing import checkMissing

In [6]:
datalib = getClient("OutFront")

In [7]:
df = pd.read_csv(datalib+"2a_dates.csv")

# Convert their date format to a standard date
df["date"] = pd.to_datetime(df["date"])

In [8]:
# A panel is missing hours
def addHours(idx, missing):
    d = {}
    d["panel"]      = idx[0]
    d["date"]       = idx[1]
    d["population"] = None
    d["hour"]       = missing
    imputed = pd.DataFrame.from_dict(d)
    imputed = imputed.set_index(["panel", "date"])
    return imputed

In [9]:
recBefore = df.shape[0]
popBefore = df["population"].sum()

# Check each panel/date for missing hours
df = df.set_index(["panel", "date"])
grp = df.groupby(level=[0,1])

count = 0
start = time.time()

for idx, val in grp:
    count += 1
    if count % 100 == 0:
        print(count)
    missing = checkMissing(val, "hour")
    if len(missing) > 0:
        imputed = addHours(idx, missing)
        df = df.append(imputed)
end = time.time()
print("{:.0f} elapsed minutes".format((end-start)/60))

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000


In [10]:
# Make sure population stayed the same (population of added records is None)
popAfter = df["population"].sum()
assert (popBefore - popAfter == 0)

In [11]:
# Make sure # of records added is a multiple of 24 (for hours / day)
recAfter = df.shape[0]
print("{:,.0f} records added".format(recAfter - recBefore))

72,810 records added


In [12]:
df = df.reset_index()
df = df.set_index(["panel", "date", "hour"])
df = df.sort_index()
df.to_csv(datalib+"2NEWb_hours.csv")